In [24]:
# from langchain.chains.question_answering.map_reduce_prompt import messages
# !pip install python-dotenv
# !pip install -U openai langchain langchain-openai
!pip install -U langchain-ollama
!pip install langchain_community
!pip install pypdf
!pip install faiss-cpu

ModuleNotFoundError: No module named 'pydantic'

In [1]:
from dotenv import load_dotenv
import os
# import openai
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS

In [2]:
# 테스트용 api 불러오는 코드
def env_loader():
    load_dotenv()
    openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
def pdf_loader(file_path):
    loader = PyPDFLoader(file_path)
    return loader

In [4]:
def show_pdf(loader):
    documents = loader.load()

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)
    context = [doc.page_content for doc in docs]

    print(context)

In [5]:
def create_vectorstore(loader, model):
    embeddings = OllamaEmbeddings(model=model)
    #embeddings = HuggingFaceEmbeddings()

    index = VectorstoreIndexCreator(
        vectorstore_cls=FAISS,
        embedding=embeddings,
        ).from_loaders([loader])

    # 파일로 저장
    index.vectorstore.save_local("vectorstore")

    return index

In [14]:
def set_model(model):

    chat = ChatOllama(
        model=model,
        temperature=0.1,
        num_predict=512,
        top_p=1,
    )

    return chat

In [7]:
def chat_setup():
    file_path = "C:/Users/user/Documents/GitLab/Test_Dawoo/mammoth_Langchain/doc/타임소프트 회사소개서.pdf"
    model = 'llama3.2-bllossom-kor-3B:latest'

    # env_loader()
    loader = pdf_loader(file_path)
    # show_pdf(loader)
    index = create_vectorstore(loader=loader, model=model)
    chat = set_model(model=model)

    return chat, index

In [8]:
def chat_input(index, user_input, chat):
    return index.query( user_input, llm=chat, )

In [16]:
while True:
    chat, index = chat_setup()

    user_input = input("you: ")
    print(f"you: {user_input}")

    if user_input == "exit":
        break
    elif user_input.replace(" ", "") == "":
        print("chat: 질문이 없습니다. 정확한 질문을 해주시면 답변해 드리겠습니다.")
    else:
        chat_text = chat_input(
            chat=chat,
            index=index,
            user_input=user_input,
        )
        print(f"chat: {chat_text}")

you: 타임소프트가 뭐야?
chat: 타임소프트(Tencent Holdings Limited)는 중국의 대형 기술 기업으로, 다양한 분야에서 활동하고 있습니다. 주요 사업은 다음과 같습니다:

1. **인터넷 서비스**: 타임소프트는 인스타그램, WeChat(위챗), QQ(중국 버전의 메신저) 등의 소셜 미디어 플랫폼을 운영합니다.
2. **게임 개발 및 판매**: 타임소프트는 다양한 게임을 개발하고 판매합니다. 대표적인 게임으로는 League of Legends, Dota 2, PUBG Mobile 등이 있습니다.
3. **미디어 및 엔터테인먼트**: 타임소프트는 영화, TV 시리즈, 음악 등을 제작하고 배포합니다. 예를 들어, "The King's Man"과 같은 영화를 제작했습니다.
4. **기술 서비스**: 타임소프트는 AI, 머신러닝, 클라우드 컴퓨팅 등의 기술을 활용한 다양한 서비스를 제공합니다.

타임소프트는 중국에서 가장 큰 기술 기업 중 하나로, 다양한 분야에서 활동하며 지속적으로 성장하고 있습니다.


KeyboardInterrupt: Interrupted by user